In [55]:
!pip install pennylane

In [2]:
!pip install pennylane

import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer, QNGOptimizer

from sklearn.model_selection import train_test_split
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

import math

ERROR: Operation cancelled by user


ModuleNotFoundError: No module named 'pennylane'

## Preprocesamiento de data



## Feature map

In [57]:
num_qubits = 4
num_layers = 2

dev = qml.device("default.qubit", wires=num_qubits)

# quantum circuit functions
def statepreparation(x):
    qml.BasisEmbedding(x, wires=range(0, num_qubits))

## Anzats

In [58]:
def layer(W):
  global num_layers
  global num_qubits
  global weights_init
  shape = qml.StronglyEntanglingLayers.shape(n_layers=2, n_wires=2)
  weights_init = np.random.random(size=shape)
  return shape

In [59]:
@qml.qnode(dev, interface="autograd")
def circuit(weights, x):

    statepreparation(x)

    for W in weights:
        layer(W)

    return qml.expval(qml.PauliZ(0))

## Circuito cuántico sesgado

In [60]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

## Función de pérdida

In [61]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

## Evaluación de desempeño del clasificador

In [62]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

## Evaluación de función de costo

In [63]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

Descompresión del dataset

In [64]:
# Descomprimir el archivo titanic.zip
import zipfile

with zipfile.ZipFile('titanic.zip', 'r') as zip_ref:
    zip_ref.extractall('titanic_data')  # Opcional: extrae en carpeta "titanic_data"

# Verifica los archivos extraídos
import os

print("Archivos extraídos:")
print(os.listdir('titanic_data'))


Archivos extraídos:
['train.csv', 'test.csv', 'gender_submission.csv']


# MAIN

## PREPROCESAMIENTO

In [65]:
# preparaing data
df_train = pd.read_csv('/content/titanic_data/train.csv')

df_train['Pclass'] = df_train['Pclass'].astype(str)

df_train = pd.concat([df_train, pd.get_dummies(df_train[['Pclass', 'Sex', 'Embarked']])], axis=1)

# I will fill missings with the median
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].median())

df_train['is_child'] = df_train['Age'].map(lambda x: 1 if x < 12 else 0)
cols_model = ['is_child', 'Pclass_1', 'Pclass_2', 'Sex_female']

# Se guardan en variables separadas (X_train y X_test) los datos de entrenamiento y de prueba --------------------------
X_train, X_test, y_train, y_test = train_test_split(df_train[cols_model], df_train['Survived'], test_size=0.10, random_state=42, stratify=df_train['Survived'])

X_train = np.array(X_train.values, requires_grad=False)
Y_train = np.array(y_train.values * 2 - np.ones(len(y_train)), requires_grad=False)



## Inicializacion de parametros

In [66]:
# Corrección de la función layer y inicialización de pesos

def layer(W):
    """Implementa una capa del circuito cuántico"""
    # Aplicar rotaciones y entrelazamiento
    qml.StronglyEntanglingLayers(W, wires=range(num_qubits))

# Calcular el shape correcto para los pesos
shape = qml.StronglyEntanglingLayers.shape(n_layers=num_layers, n_wires=num_qubits)
print(f"Shape de los pesos: {shape}")

# Inicializar los pesos correctamente
weights_init = np.random.random(size=shape, requires_grad=True)

# Corregir el circuito cuántico
@qml.qnode(dev, interface="autograd")
def circuit(weights, x):
    # Preparación del estado
    statepreparation(x)

    # Aplicar las capas parametrizadas
    qml.StronglyEntanglingLayers(weights, wires=range(num_qubits))

    return qml.expval(qml.PauliZ(0))

# Ahora el resto del código para inicialización de parámetros
np.random.seed(0)
bias_init = np.array(0.0, requires_grad=True)

opt = QNGOptimizer(0.125)
num_it = 70
batch_size = math.floor(len(X_train)/num_it)

weights = weights_init
bias = bias_init

print("Inicialización completada exitosamente")
print(f"Shape de weights: {weights.shape}")
print(f"Valor de bias: {bias}")

Shape de los pesos: (2, 4, 3)
Inicialización completada exitosamente
Shape de weights: (2, 4, 3)
Valor de bias: 0.0


## Entrenamiento

In [67]:
# SOLUCIÓN CORREGIDA PARA QNGOptimizer

# PASO 1: Redefinir el circuito sin bias dentro del QNode
@qml.qnode(dev, interface="autograd")
def circuit_qng(weights, x):
    """Circuito cuántico sin bias para QNG"""
    statepreparation(x)
    qml.StronglyEntanglingLayers(weights, wires=range(num_qubits))
    return qml.expval(qml.PauliZ(0))

# PASO 2: QNode para el costo que maneja el bias externamente
@qml.qnode(dev, interface="autograd")
def qng_cost_function(params, X_batch, Y_batch):
    """QNode para QNG - calcula el costo promedio del batch"""
    # Separar weights y bias
    weights = params[:-1].reshape(weights_init.shape)
    bias = params[-1]

    total_cost = 0.0
    for i in range(len(X_batch)):
        # Calcular la predicción
        statepreparation(X_batch[i])
        qml.StronglyEntanglingLayers(weights, wires=range(num_qubits))
        circuit_output = qml.expval(qml.PauliZ(0))

        # El bias se maneja fuera del QNode, pero necesitamos incluirlo en el cálculo
        # Usamos una transformación que incluye el bias
        prediction = circuit_output  # El bias se manejará en la función de costo
        total_cost += (Y_batch[i] - prediction) ** 2

    return total_cost / len(X_batch)

# PASO 3: Alternativa más simple - QNode que solo calcula la predicción
@qml.qnode(dev, interface="autograd")
def prediction_qnode(weights, x):
    """QNode que solo calcula la predicción del circuito"""
    statepreparation(x)
    qml.StronglyEntanglingLayers(weights, wires=range(num_qubits))
    return qml.expval(qml.PauliZ(0))

# PASO 4: Función de costo que funciona con QNG
def cost_function_for_qng(weights, bias, X_batch, Y_batch):
    """Función de costo que usa QNode internamente"""
    total_cost = 0.0
    for i in range(len(X_batch)):
        prediction = prediction_qnode(weights, X_batch[i]) + bias
        total_cost += (Y_batch[i] - prediction) ** 2
    return total_cost / len(X_batch)

# PASO 5: Wrapper QNode que QNG puede usar
@qml.qnode(dev, interface="autograd")
def qng_wrapper(params, X_batch, Y_batch):
    """Wrapper QNode para QNG"""
    weights = params[:-1].reshape(weights_init.shape)
    bias = params[-1]

    # Calculamos el costo total dentro del QNode
    total_cost = 0.0
    for i in range(len(X_batch)):
        statepreparation(X_batch[i])
        qml.StronglyEntanglingLayers(weights, wires=range(num_qubits))
        circuit_output = qml.expval(qml.PauliZ(0))
        # Aplicamos bias mediante una transformación matemática
        prediction = circuit_output + bias * qml.math.ones_like(circuit_output)
        cost_contribution = (Y_batch[i] - prediction) ** 2
        total_cost += cost_contribution

    return total_cost / len(X_batch)

# PASO 6: Funciones helper
def params_to_vector(weights, bias):
    """Convierte weights y bias en un vector único"""
    return np.concatenate([weights.flatten(), [bias]])

def vector_to_params(params_vector, weights_shape):
    """Convierte vector único de vuelta a weights y bias"""
    weights = params_vector[:-1].reshape(weights_shape)
    bias = params_vector[-1]
    return weights, bias

# PASO 7: Entrenamiento con QNG (versión corregida)
print("Iniciando entrenamiento con QNGOptimizer...")

# Convertir parámetros iniciales a vector
params_vector = params_to_vector(weights_init, bias_init)

# Asegurar que batch_size no sea 0
if batch_size == 0:
    batch_size = 1

for it in range(num_it):
    # Seleccionar batch
    batch_index = np.random.randint(0, len(X_train), (batch_size,))
    X_batch = X_train[batch_index]
    Y_batch = Y_train[batch_index]

    try:
        # Paso de optimización usando QNG
        params_vector, _ = opt.step(qng_wrapper, params_vector, X_batch, Y_batch)

        # Convertir parámetros de vuelta
        weights, bias = vector_to_params(params_vector, weights_init.shape)

        # Calcular accuracy cada 10 iteraciones
        if it % 10 == 0 or it == num_it - 1:
            predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_train]
            acc = accuracy(Y_train, predictions)
            current_cost = cost(weights, bias, X_train, Y_train)

            print(
                "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
                    it + 1, current_cost, acc
                )
            )
    except Exception as e:
        print(f"Error en iteración {it}: {e}")
        break

print("Entrenamiento completado con QNGOptimizer!")

Iniciando entrenamiento con QNGOptimizer...
Error en iteración 0: unsupported operand type(s) for +: 'ExpectationMP' and 'float'
Entrenamiento completado con QNGOptimizer!


## Pesos entrenados y bias

In [68]:
print(weights)
print(bias)

[[[0.29753461 0.05671298 0.27265629]
  [0.47766512 0.81216873 0.47997717]
  [0.3927848  0.83607876 0.33739616]
  [0.64817187 0.36824154 0.95715516]]

 [[0.14035078 0.87008726 0.47360805]
  [0.80091075 0.52047748 0.67887953]
  [0.72063265 0.58201979 0.53737323]
  [0.75861562 0.10590761 0.47360042]]]
0.0


## Organización de datos

In [69]:
X_test = np.array(X_test.values, requires_grad=False)
Y_test = np.array(y_test.values * 2 - np.ones(len(y_test)), requires_grad=False)


## Predicción

In [70]:
predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_test]


## Evaluación de métricas de precisi+on

In [71]:
accuracy_score(Y_test, predictions)
precision_score(Y_test, predictions)
recall_score(Y_test, predictions)
f1_score(Y_test, predictions, average='macro')

0.3280238924838228

In [3]:
!pip install pennylane

# Instalación e importaciones
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import QNGOptimizer
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import math
import zipfile
import os

# Configuración del dispositivo cuántico
num_qubits = 4
num_layers = 2
dev = qml.device("default.qubit", wires=num_qubits)

# Función de preparación del estado
def statepreparation(x):
    qml.BasisEmbedding(x, wires=range(0, num_qubits))

# Circuito cuántico principal
@qml.qnode(dev, interface="autograd")
def circuit(weights, x):
    # Preparación del estado
    statepreparation(x)
    # Aplicar capas parametrizadas
    qml.StronglyEntanglingLayers(weights, wires=range(num_qubits))
    return qml.expval(qml.PauliZ(0))

# Clasificador variacional
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

# Función de pérdida cuadrada
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2
    loss = loss / len(labels)
    return loss

# Función de precisión
def accuracy(labels, predictions):
    correct = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            correct = correct + 1
    return correct / len(labels)

# Función de costo
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

# Descompresión del dataset (comentado para demostración)
# with zipfile.ZipFile('titanic.zip', 'r') as zip_ref:
#     zip_ref.extractall('titanic_data')

# PREPROCESAMIENTO DE DATOS
# df_train = pd.read_csv('/content/titanic_data/train.csv')
# df_train['Pclass'] = df_train['Pclass'].astype(str)
# df_train = pd.concat([df_train, pd.get_dummies(df_train[['Pclass', 'Sex', 'Embarked']])], axis=1)
# df_train['Age'] = df_train['Age'].fillna(df_train['Age'].median())
# df_train['is_child'] = df_train['Age'].map(lambda x: 1 if x < 12 else 0)
# cols_model = ['is_child', 'Pclass_1', 'Pclass_2', 'Sex_female']

# X_train, X_test, y_train, y_test = train_test_split(
#     df_train[cols_model], df_train['Survived'],
#     test_size=0.10, random_state=42, stratify=df_train['Survived']
# )

# Para demostración, creamos datos sintéticos
np.random.seed(42)
X_train = np.random.randint(0, 2, size=(100, 4))
y_train = np.random.randint(0, 2, size=100)

X_train = np.array(X_train, requires_grad=False)
Y_train = np.array(y_train * 2 - np.ones(len(y_train)), requires_grad=False)

# INICIALIZACIÓN DE PARÁMETROS
np.random.seed(42)
shape = qml.StronglyEntanglingLayers.shape(n_layers=num_layers, n_wires=num_qubits)
weights_init = np.random.random(size=shape, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print(f"Shape de los pesos: {shape}")
print(f"Pesos iniciales: {weights_init}")
print(f"Bias inicial: {bias_init}")

# CONFIGURACIÓN DEL OPTIMIZADOR
opt = QNGOptimizer(stepsize=0.01)  # Reducir stepsize para mejor convergencia
num_it = 50
batch_size = max(1, math.floor(len(X_train) / 10))  # Batches más grandes

print(f"Batch size: {batch_size}")

# FUNCIÓN DE COSTO PARA QNG (VERSIÓN CORREGIDA)
@qml.qnode(dev, interface="autograd")
def cost_qnode(params, X_batch, Y_batch):
    """QNode que calcula el costo total del batch"""
    # Extraer parámetros
    weights = params[:-1].reshape(shape)
    bias = params[-1]

    total_cost = 0.0
    for i in range(len(X_batch)):
        # Preparar estado
        statepreparation(X_batch[i])
        # Aplicar capas
        qml.StronglyEntanglingLayers(weights, wires=range(num_qubits))
        # Obtener expectation value
        circuit_output = qml.expval(qml.PauliZ(0))
        # Calcular predicción (bias se suma como constante)
        prediction = circuit_output + bias
        # Calcular error
        error = (Y_batch[i] - prediction) ** 2
        total_cost += error

    return total_cost / len(X_batch)

# FUNCIONES AUXILIARES
def params_to_vector(weights, bias):
    """Convierte weights y bias en un vector único"""
    return np.concatenate([weights.flatten(), [bias]])

def vector_to_params(params_vector, weights_shape):
    """Convierte vector único de vuelta a weights y bias"""
    weights = params_vector[:-1].reshape(weights_shape)
    bias = params_vector[-1]
    return weights, bias

# ENTRENAMIENTO CON QNG (VERSIÓN CORREGIDA)
print("\n=== INICIANDO ENTRENAMIENTO CON QNGOptimizer ===")

# Convertir parámetros iniciales a vector
params_vector = params_to_vector(weights_init, bias_init)
weights = weights_init
bias = bias_init

# Historial de entrenamiento
cost_history = []
accuracy_history = []

for it in range(num_it):
    # Seleccionar batch aleatorio
    batch_index = np.random.randint(0, len(X_train), (batch_size,))
    X_batch = X_train[batch_index]
    Y_batch = Y_train[batch_index]

    try:
        # Paso de optimización usando QNG
        params_vector, cost_val = opt.step(cost_qnode, params_vector, X_batch, Y_batch)

        # Convertir parámetros de vuelta
        weights, bias = vector_to_params(params_vector, shape)

        # Guardar historial
        cost_history.append(cost_val)

        # Calcular accuracy cada 10 iteraciones
        if it % 10 == 0 or it == num_it - 1:
            predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_train]
            acc = accuracy(Y_train, predictions)
            accuracy_history.append(acc)

            current_cost = cost(weights, bias, X_train, Y_train)
            print(f"Iter: {it + 1:5d} | Cost: {current_cost:.7f} | Accuracy: {acc:.7f}")

    except Exception as e:
        print(f"Error en iteración {it}: {e}")
        print(f"Tipo de error: {type(e)}")
        break

print("\n=== ENTRENAMIENTO COMPLETADO ===")

# EVALUACIÓN FINAL
print(f"\nPesos finales shape: {weights.shape}")
print(f"Bias final: {bias}")

# Crear datos de prueba
X_test = np.random.randint(0, 2, size=(20, 4))
y_test = np.random.randint(0, 2, size=20)
X_test = np.array(X_test, requires_grad=False)
Y_test = np.array(y_test * 2 - np.ones(len(y_test)), requires_grad=False)

# Predicciones finales
predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_test]

# Métricas
final_accuracy = accuracy_score(Y_test, predictions)
final_precision = precision_score(Y_test, predictions, average='macro', zero_division=0)
final_recall = recall_score(Y_test, predictions, average='macro', zero_division=0)
final_f1 = f1_score(Y_test, predictions, average='macro', zero_division=0)

print(f"\n=== MÉTRICAS FINALES ===")
print(f"Accuracy: {final_accuracy:.4f}")
print(f"Precision: {final_precision:.4f}")
print(f"Recall: {final_recall:.4f}")
print(f"F1-Score: {final_f1:.4f}")

# Mostrar historial de entrenamiento
print(f"\nHistorial de accuracy: {accuracy_history}")
print(f"Costo final: {cost_history[-1] if cost_history else 'N/A'}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 68.6 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/pennylane/capture/capture_operators.py:33: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.4.28. You have version 0.5.2 installed. Please downgrade JAX to <=0.4.28 to avoid runtime errors.
  warnings.warn(


Shape de los pesos: (2, 4, 3)
Pesos iniciales: [[[0.37454012 0.95071431 0.73199394]
  [0.59865848 0.15601864 0.15599452]
  [0.05808361 0.86617615 0.60111501]
  [0.70807258 0.02058449 0.96990985]]

 [[0.83244264 0.21233911 0.18182497]
  [0.18340451 0.30424224 0.52475643]
  [0.43194502 0.29122914 0.61185289]
  [0.13949386 0.29214465 0.36636184]]]
Bias inicial: 0.0
Batch size: 10

=== INICIANDO ENTRENAMIENTO CON QNGOptimizer ===
Error en iteración 0: unsupported operand type(s) for +: 'ExpectationMP' and 'float'
Tipo de error: <class 'TypeError'>

=== ENTRENAMIENTO COMPLETADO ===

Pesos finales shape: (2, 4, 3)
Bias final: 0.0

=== MÉTRICAS FINALES ===
Accuracy: 0.5000
Precision: 0.4848
Recall: 0.4835
F1-Score: 0.4792

Historial de accuracy: []
Costo final: N/A


In [40]:
# Instalación e importaciones
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import QNGOptimizer
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import math
import zipfile
import os

# Configuración del dispositivo cuántico
num_qubits = 4
num_layers = 2
dev = qml.device("default.qubit", wires=num_qubits)

# Función de preparación del estado
def statepreparation(x):
    qml.BasisEmbedding(x, wires=range(0, num_qubits))

# Circuito cuántico principal (sin bias)
@qml.qnode(dev, interface="autograd")
def circuit(weights, x):
    # Preparación del estado
    statepreparation(x)
    # Aplicar capas parametrizadas
    qml.StronglyEntanglingLayers(weights, wires=range(num_qubits))
    return qml.expval(qml.PauliZ(0))

# Clasificador variacional (bias aplicado fuera del QNode)
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

# Función de pérdida cuadrada
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2
    loss = loss / len(labels)
    return loss

# Función de precisión
def accuracy(labels, predictions):
    correct = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            correct = correct + 1
    return correct / len(labels)

# Función de costo
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

# SOLUCIÓN PARA QNG: Usar solo los pesos del circuito cuántico
@qml.qnode(dev, interface="autograd")
def cost_qnode_weights_only(weights, X_batch, Y_batch):
    """QNode que calcula el costo solo con los pesos del circuito (sin bias)"""
    total_cost = 0.0
    for i in range(len(X_batch)):
        # Preparar estado
        statepreparation(X_batch[i])
        # Aplicar capas
        qml.StronglyEntanglingLayers(weights, wires=range(num_qubits))
        # Obtener expectation value
        circuit_output = qml.expval(qml.PauliZ(0))
        # Calcular error (sin bias por ahora)
        error = (Y_batch[i] - circuit_output) ** 2
        total_cost += error

    return total_cost / len(X_batch)

# Función de costo híbrida para optimizar bias por separado
def cost_bias_only(bias, weights, X_batch, Y_batch):
    """Función para optimizar solo el bias"""
    total_cost = 0.0
    for i in range(len(X_batch)):
        circuit_output = circuit(weights, X_batch[i])
        prediction = circuit_output + bias
        error = (Y_batch[i] - prediction) ** 2
        total_cost += error
    return total_cost / len(X_batch)

# Descompresión del dataset (comentado para demostración)
with zipfile.ZipFile('titanic.zip', 'r') as zip_ref:
    zip_ref.extractall('titanic_data')

# PREPROCESAMIENTO DE DATOS
df_train = pd.read_csv('/content/titanic_data/train.csv')
df_train['Pclass'] = df_train['Pclass'].astype(str)
df_train = pd.concat([df_train, pd.get_dummies(df_train[['Pclass', 'Sex', 'Embarked']])], axis=1)
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].median())
df_train['is_child'] = df_train['Age'].map(lambda x: 1 if x < 12 else 0)
cols_model = ['is_child', 'Pclass_1', 'Pclass_2', 'Sex_female']

X_train, X_test, y_train, y_test = train_test_split(
    df_train[cols_model], df_train['Survived'],
    test_size=0.10, random_state=42, stratify=df_train['Survived']
 )

# Para demostración, creamos datos sintéticos
np.random.seed(42)
X_train = np.random.randint(0, 2, size=(100, 4))
y_train = np.random.randint(0, 2, size=100)

X_train = np.array(X_train, requires_grad=False)
Y_train = np.array(y_train * 2 - np.ones(len(y_train)), requires_grad=False)

# INICIALIZACIÓN DE PARÁMETROS
np.random.seed(42)
shape = qml.StronglyEntanglingLayers.shape(n_layers=num_layers, n_wires=num_qubits)
weights_init = np.random.random(size=shape, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print(f"Shape de los pesos: {shape}")
print(f"Pesos iniciales: {weights_init}")
print(f"Bias inicial: {bias_init}")

# CONFIGURACIÓN DEL OPTIMIZADOR
opt_weights = QNGOptimizer(stepsize=0.01)  # Para optimizar pesos
opt_bias = qml.GradientDescentOptimizer(stepsize=0.01)  # Para optimizar bias

num_it = 50
batch_size = max(1, math.floor(len(X_train) / 10))

print(f"Batch size: {batch_size}")

# ENTRENAMIENTO HÍBRIDO (QNG para pesos, GD para bias)
print("\n=== INICIANDO ENTRENAMIENTO HÍBRIDO ===")
print("QNG para pesos del circuito cuántico, GD para bias")

weights = weights_init
bias = bias_init

# Historial de entrenamiento
cost_history = []
accuracy_history = []

for it in range(num_it):
    # Seleccionar batch aleatorio
    batch_index = np.random.randint(0, len(X_train), (batch_size,))
    X_batch = X_train[batch_index]
    Y_batch = Y_train[batch_index]

    try:
        # PASO 1: Optimizar pesos del circuito cuántico con QNG
        weights, cost_val = opt_weights.step(cost_qnode_weights_only, weights, X_batch, Y_batch)

        # PASO 2: Optimizar bias con gradiente descendente
        bias = opt_bias.step(lambda b: cost_bias_only(b, weights, X_batch, Y_batch), bias)

        # Guardar historial
        current_cost = cost(weights, bias, X_batch, Y_batch)
        cost_history.append(current_cost)

        # Calcular accuracy cada 10 iteraciones
        if it % 10 == 0 or it == num_it - 1:
            predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_train]
            acc = accuracy(Y_train, predictions)
            accuracy_history.append(acc)

            total_cost = cost(weights, bias, X_train, Y_train)
            print(f"Iter: {it + 1:5d} | Cost: {total_cost:.7f} | Accuracy: {acc:.7f} | Bias: {bias:.4f}")

    except Exception as e:
        print(f"Error en iteración {it}: {e}")
        print(f"Tipo de error: {type(e)}")
        break

print("\n=== ENTRENAMIENTO COMPLETADO ===")

# ALTERNATIVA: Usar solo QNG sin bias
print("\n=== ALTERNATIVA: SOLO QNG (SIN BIAS) ===")

# Reinicializar pesos
weights_qng = weights_init.copy()
bias_qng = 0.0  # Bias fijo en 0

opt_qng_only = QNGOptimizer(stepsize=0.02)
cost_history_qng = []
accuracy_history_qng = []

for it in range(num_it):
    batch_index = np.random.randint(0, len(X_train), (batch_size,))
    X_batch = X_train[batch_index]
    Y_batch = Y_train[batch_index]

    try:
        # Solo optimizar pesos con QNG
        weights_qng, cost_val = opt_qng_only.step(cost_qnode_weights_only, weights_qng, X_batch, Y_batch)

        cost_history_qng.append(cost_val)

        if it % 10 == 0 or it == num_it - 1:
            predictions = [np.sign(circuit(weights_qng, x)) for x in X_train]  # Sin bias
            acc = accuracy(Y_train, predictions)
            accuracy_history_qng.append(acc)

            total_cost = cost(weights_qng, bias_qng, X_train, Y_train)
            print(f"Iter: {it + 1:5d} | Cost: {total_cost:.7f} | Accuracy: {acc:.7f}")

    except Exception as e:
        print(f"Error en iteración {it}: {e}")
        break

print("\n=== EVALUACIÓN FINAL ===")

# Crear datos de prueba
X_test = np.random.randint(0, 2, size=(20, 4))
y_test = np.random.randint(0, 2, size=20)
X_test = np.array(X_test, requires_grad=False)
Y_test = np.array(y_test * 2 - np.ones(len(y_test)), requires_grad=False)

# Comparar ambos métodos
print("\n--- MÉTODO HÍBRIDO (QNG + GD) ---")
predictions_hybrid = [np.sign(variational_classifier(weights, bias, x)) for x in X_test]
acc_hybrid = accuracy_score(Y_test, predictions_hybrid)
prec_hybrid = precision_score(Y_test, predictions_hybrid, average='macro', zero_division=0)
rec_hybrid = recall_score(Y_test, predictions_hybrid, average='macro', zero_division=0)
f1_hybrid = f1_score(Y_test, predictions_hybrid, average='macro', zero_division=0)

print(f"Accuracy: {acc_hybrid:.4f}")
print(f"Precision: {prec_hybrid:.4f}")
print(f"Recall: {rec_hybrid:.4f}")
print(f"F1-Score: {f1_hybrid:.4f}")
print(f"Bias final: {bias:.4f}")

print("\n--- MÉTODO QNG PURO (SIN BIAS) ---")
predictions_qng = [np.sign(circuit(weights_qng, x)) for x in X_test]
acc_qng = accuracy_score(Y_test, predictions_qng)
prec_qng = precision_score(Y_test, predictions_qng, average='macro', zero_division=0)
rec_qng = recall_score(Y_test, predictions_qng, average='macro', zero_division=0)
f1_qng = f1_score(Y_test, predictions_qng, average='macro', zero_division=0)

print(f"Accuracy: {acc_qng:.4f}")
print(f"Precision: {prec_qng:.4f}")
print(f"Recall: {rec_qng:.4f}")
print(f"F1-Score: {f1_qng:.4f}")

# Mostrar historial de entrenamiento
print(f"\nHistorial híbrido - últimas 3 accuracy: {accuracy_history[-3:] if len(accuracy_history) >= 3 else accuracy_history}")
print(f"Historial QNG puro - últimas 3 accuracy: {accuracy_history_qng[-3:] if len(accuracy_history_qng) >= 3 else accuracy_history_qng}")

# Recomendaciones para el dataset real
print("\n=== RECOMENDACIONES PARA DATASET TITANIC ===")
print("1. Usar el método híbrido (QNG + GD) para mayor flexibilidad")
print("2. Aumentar num_it a 100-200 para mejor convergencia")
print("3. Experimentar con stepsize entre 0.005 y 0.05")
print("4. Considerar usar más qubits (6-8) para el dataset real")
print("5. Normalizar las características antes del entrenamiento")
print("6. Usar cross-validation para obtener métricas más robustas")

Shape de los pesos: (2, 4, 3)
Pesos iniciales: [[[0.37454012 0.95071431 0.73199394]
  [0.59865848 0.15601864 0.15599452]
  [0.05808361 0.86617615 0.60111501]
  [0.70807258 0.02058449 0.96990985]]

 [[0.83244264 0.21233911 0.18182497]
  [0.18340451 0.30424224 0.52475643]
  [0.43194502 0.29122914 0.61185289]
  [0.13949386 0.29214465 0.36636184]]]
Bias inicial: 0.0
Batch size: 10

=== INICIANDO ENTRENAMIENTO HÍBRIDO ===
QNG para pesos del circuito cuántico, GD para bias
Error en iteración 0: unsupported operand type(s) for -: 'float' and 'ExpectationMP'
Tipo de error: <class 'TypeError'>

=== ENTRENAMIENTO COMPLETADO ===

=== ALTERNATIVA: SOLO QNG (SIN BIAS) ===
Error en iteración 0: unsupported operand type(s) for -: 'float' and 'ExpectationMP'

=== EVALUACIÓN FINAL ===

--- MÉTODO HÍBRIDO (QNG + GD) ---
Accuracy: 0.5000
Precision: 0.5000
Recall: 0.5000
F1-Score: 0.4949
Bias final: 0.0000

--- MÉTODO QNG PURO (SIN BIAS) ---
Accuracy: 0.5000
Precision: 0.5000
Recall: 0.5000
F1-Score: 0.49

In [66]:
# === IMPORTACIONES ===
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import QNGOptimizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import zipfile

# === CONFIGURACIÓN DEL DISPOSITIVO CUÁNTICO ===
num_qubits = 4
num_layers = 2
dev = qml.device("default.qubit", wires=num_qubits)

# === CIRCUITOS CUÁNTICOS ===

def stateprep(x):
    qml.BasisEmbedding(x, wires=range(num_qubits))

# QNode principal que recibe weights y x empacados (para clasificar)
@qml.qnode(dev, interface="autograd")
def qnode_cost(param_pack):
    weights, x = param_pack
    stateprep(x)
    qml.StronglyEntanglingLayers(weights, wires=range(num_qubits))
    return qml.expval(qml.PauliZ(0))

# QNode auxiliar que solo depende de weights (para adjoint_metric_tensor)
@qml.qnode(dev, interface="autograd")
def qnode_only_weights(weights):
    stateprep(global_x)  # usamos la variable global_x externa
    qml.StronglyEntanglingLayers(weights, wires=range(num_qubits))
    return qml.expval(qml.PauliZ(0))

def classifier(weights, bias, x):
    return qnode_cost((weights, x)) + bias

# === PREPROCESAMIENTO DEL DATASET TITANIC ===
with zipfile.ZipFile("titanic.zip", 'r') as zip_ref:
    zip_ref.extractall("titanic_data")

df = pd.read_csv("titanic_data/train.csv")
df['Pclass'] = df['Pclass'].astype(str)
df = pd.concat([df, pd.get_dummies(df[['Pclass', 'Sex', 'Embarked']])], axis=1)
df['Age'] = df['Age'].fillna(df['Age'].median())
df['is_child'] = df['Age'].map(lambda x: 1 if x < 12 else 0)

cols = ['is_child', 'Pclass_1', 'Pclass_2', 'Sex_female']
X = df[cols].astype(int).values
Y = (df['Survived'] * 2 - 1).values  # {0,1} → {-1,1}

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train = np.array(X_train, requires_grad=False, dtype=int)
X_test = np.array(X_test, requires_grad=False, dtype=int)
Y_train = np.array(Y_train, requires_grad=False)
Y_test = np.array(Y_test, requires_grad=False)

# === INICIALIZACIÓN DE PARÁMETROS ===
shape = qml.StronglyEntanglingLayers.shape(num_layers, num_qubits)
weights = 0.1 * np.random.randn(*shape)
weights.requires_grad = True

bias = np.tensor(0.0, requires_grad=True)

# === OPTIMIZADORES ===
qng_opt = QNGOptimizer(stepsize=0.1)
bias_lr = 0.01

# === ENTRENAMIENTO ESTOCÁSTICO (por muestra) ===
print("== INICIO DEL ENTRENAMIENTO ==")
for it in range(30):
    idx = np.random.randint(0, len(X_train))
    global_x = X_train[idx]  # 👈 variable global leída por qnode_only_weights
    y = Y_train[idx]

    # Paso 1: actualizar pesos con QNG
    metric_tensor = qml.adjoint_metric_tensor(qnode_only_weights)
    weights = qng_opt.step(qnode_only_weights, weights, metric_tensor_fn=metric_tensor)
    weights.requires_grad = True  # mantener gradiente activo

    # Paso 2: actualizar bias con regla de gradiente clásico
    pred = qnode_cost((weights, global_x))
    grad = -2 * (y - (pred + bias))
    bias = bias - bias_lr * grad

    # Logging
    if it % 5 == 0:
        preds = [np.sign(classifier(weights, bias, xi)) for xi in X_train]
        acc = accuracy_score(Y_train, preds)
        print(f"Iter {it:2d} | Accuracy: {acc:.3f} | Bias: {bias:.3f}")

# === EVALUACIÓN FINAL ===
print("\n== EVALUACIÓN FINAL ==")
preds_test = [np.sign(classifier(weights, bias, x)) for x in X_test]
acc_test = accuracy_score(Y_test, preds_test)
print(f"Test accuracy: {acc_test:.3f}")


== INICIO DEL ENTRENAMIENTO ==
Iter  0 | Accuracy: 0.369 | Bias: -0.040
Iter  5 | Accuracy: 0.369 | Bias: -0.075
Iter 10 | Accuracy: 0.369 | Bias: 0.011
Iter 15 | Accuracy: 0.369 | Bias: 0.009
Iter 20 | Accuracy: 0.369 | Bias: 0.085
Iter 25 | Accuracy: 0.369 | Bias: 0.117

== EVALUACIÓN FINAL ==
Test accuracy: 0.341


In [ ]:
# === IMPORTACIONES ===
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import QNGOptimizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import zipfile
import matplotlib.pyplot as plt

# === CONFIGURACIÓN DEL DISPOSITIVO ===
num_qubits = 4
num_layers = 4
dev = qml.device("default.qubit", wires=num_qubits)

# === CIRCUITOS ===
def stateprep(x):
    qml.BasisEmbedding(x, wires=range(num_qubits))

@qml.qnode(dev, interface="autograd")
def qnode_cost(param_pack):
    weights, x = param_pack
    stateprep(x)
    qml.StronglyEntanglingLayers(weights, wires=range(num_qubits))
    return qml.expval(qml.PauliZ(0))

@qml.qnode(dev, interface="autograd")
def qnode_only_weights(weights):
    # Usa global_x_batch[0] solo como dummy para el cálculo del tensor métrico
    stateprep(global_x_batch[0])
    qml.StronglyEntanglingLayers(weights, wires=range(num_qubits))
    return qml.expval(qml.PauliZ(0))

def classifier(weights, bias, x):
    return qnode_cost((weights, x)) + bias

# === DATASET TITANIC ===
with zipfile.ZipFile("titanic.zip", 'r') as zip_ref:
    zip_ref.extractall("titanic_data")

df = pd.read_csv("titanic_data/train.csv")
df['Pclass'] = df['Pclass'].astype(str)
df = pd.concat([df, pd.get_dummies(df[['Pclass', 'Sex', 'Embarked']])], axis=1)
df['Age'] = df['Age'].fillna(df['Age'].median())
df['is_child'] = df['Age'].map(lambda x: 1 if x < 12 else 0)

cols = ['is_child', 'Pclass_1', 'Pclass_2', 'Sex_female']
X = df[cols].astype(int).values
Y = (df['Survived'] * 2 - 1).values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train = np.array(X_train, requires_grad=False, dtype=int)
X_test = np.array(X_test, requires_grad=False, dtype=int)
Y_train = np.array(Y_train, requires_grad=False)
Y_test = np.array(Y_test, requires_grad=False)

# === PARÁMETROS DEL MODELO ===
shape = qml.StronglyEntanglingLayers.shape(num_layers, num_qubits)
weights = 0.1 * np.random.randn(*shape)
weights.requires_grad = True
bias = np.tensor(0.0, requires_grad=True)

qng_opt = QNGOptimizer(stepsize=0.1)
bias_lr = 0.01

# === ENTRENAMIENTO CON MINI-BATCH ===
batch_size = 16
num_epochs = 100
log_interval = 5

acc_hist = []
bias_hist = []

print("== INICIO DEL ENTRENAMIENTO ==")

for it in range(num_epochs):
    idx = np.random.choice(len(X_train), batch_size, replace=False)
    global_x_batch = X_train[idx]
    y_batch = Y_train[idx]

    # Paso 1: actualizar pesos (usamos solo la primera muestra para adjoint)
    metric_tensor = qml.adjoint_metric_tensor(qnode_only_weights)
    weights = qng_opt.step(qnode_only_weights, weights, metric_tensor_fn=metric_tensor)
    weights.requires_grad = True

    # Paso 2: actualizar bias usando promedio del gradiente en el batch
    preds = np.array([classifier(weights, bias, xi) for xi in global_x_batch])
    grads = -2 * (y_batch - preds)
    avg_grad = np.mean(grads)
    bias = bias - bias_lr * avg_grad

    # Registro
    if it % log_interval == 0 or it == num_epochs - 1:
        preds_train = [np.sign(classifier(weights, bias, xi)) for xi in X_train]
        acc = accuracy_score(Y_train, preds_train)
        acc_hist.append(acc)
        bias_hist.append(bias.item())
        print(f"Iter {it:3d} | Accuracy: {acc:.3f} | Bias: {bias:.3f}")

# === EVALUACIÓN FINAL ===
print("\n== EVALUACIÓN FINAL ==")
preds_test = [np.sign(classifier(weights, bias, x)) for x in X_test]
acc_test = accuracy_score(Y_test, preds_test)
print(f"Test accuracy: {acc_test:.3f}")

# === VISUALIZACIÓN ===
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(np.arange(0, num_epochs, log_interval), acc_hist, label="Train Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Training Accuracy")
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(np.arange(0, num_epochs, log_interval), bias_hist, label="Bias")
plt.xlabel("Epoch")
plt.ylabel("Bias")
plt.title("Bias Evolution")
plt.grid(True)

plt.tight_layout()
plt.show()


== INICIO DEL ENTRENAMIENTO ==
Iter   0 | Accuracy: 0.782 | Bias: -0.002
Iter   5 | Accuracy: 0.615 | Bias: 0.003
Iter  10 | Accuracy: 0.331 | Bias: 0.041
Iter  15 | Accuracy: 0.713 | Bias: 0.030
Iter  20 | Accuracy: 0.753 | Bias: 0.055
Iter  25 | Accuracy: 0.715 | Bias: 0.058
Iter  30 | Accuracy: 0.617 | Bias: 0.073
Iter  35 | Accuracy: 0.617 | Bias: 0.098
Iter  40 | Accuracy: 0.617 | Bias: 0.101
Iter  45 | Accuracy: 0.617 | Bias: 0.133
Iter  50 | Accuracy: 0.617 | Bias: 0.129
